In [ ]:
!pip install neo4j yfiles_jupyter_graphs_for_neo4j pandas

In [ ]:
from neo4j import GraphDatabase

In [ ]:
from google.colab import userdata
neo4j_password = userdata.get('neo4j_password')

driver = GraphDatabase.driver("neo4j+s://7ffc03b9.databases.neo4j.io:7687", auth=("neo4j", neo4j_password))

In [ ]:
def insert_data(tx):
    query = """
      CREATE (A:Location {name: 'A'})
      CREATE (B:Location {name: 'B'})
      CREATE (C:Location {name: 'C'})
      CREATE (D:Location {name: 'D'})

      CREATE (A)-[:CONNECTED {weight: 1}]->(B)
      CREATE (A)-[:CONNECTED {weight: 2}]->(C)
      CREATE (A)-[:CONNECTED {weight: 4}]->(D)
      CREATE (B)-[:CONNECTED {weight: 2}]->(C)
      CREATE (B)-[:CONNECTED {weight: 3}]->(D)
      CREATE (C)-[:CONNECTED {weight: 1}]->(D)

    RETURN *
    """
    tx.run(query)

with driver.session() as session:
    session.execute_write(insert_data)

In [ ]:
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget

g = Neo4jGraphWidget(driver)

def show_graph(driver):
    query = """
    MATCH (n)-[r]->(m)
    RETURN n, r, m
    """
    g.show_cypher(query)

show_graph(driver)

GraphWidget(layout=Layout(height='740px', width='100%'))

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [ ]:
import pandas as pd

def run_dijkstra(driver):
    query = """
      WITH 'A' AS startName, 'D' AS endName
      MATCH (start:Location {name: startName}), (end:Location {name: endName})
      CALL apoc.algo.dijkstra(start, end, 'CONNECTED', 'weight') YIELD path, weight
      RETURN
        nodes(path) AS nodes,
        relationships(path) AS edges,
        weight
    """
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

results = run_dijkstra(driver)
pd.DataFrame(results)

,nodes,edges,weight
0,"[{'name': 'A'}, {'name': 'C'}, {'name': 'D'}]","[({'name': 'A'}, CONNECTED, {'name': 'C'}), ({...",3.0
